# Masking

This code demonstrates the reduction in the number of calculations when using a `mask` on a pipeline statement.

In [1]:
from quantopian.pipeline.filters import StaticAssets
from quantopian.pipeline.data import USEquityPricing
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
start_date = '2018-08-02'
end_date = '2018-08-02' 
tickers = ['GOOG']

In [4]:
from quantopian.pipeline.factors import CustomFactor

def universe():
    return StaticAssets(symbols(tickers))

class BogusFactor(CustomFactor):
    inputs=[USEquityPricing.close]
    def compute(self, today, assets, out, p):
        print(p.shape)
        out[:] = p[-1]


u = universe()
bogus = BogusFactor(window_length=100)
pipe = Pipeline(
    columns={
        'b': bogus
    },
    screen=u
)
mdf = run_pipeline(pipe, start_date, end_date)
mdf

(100, 8652)


,,b
2018-08-02 00:00:00+00:00,Equity(46631 [GOOG]),1219.08


Note that the shape of `p` input above is 100 rows (for 100 days of `window_length`) x 8652 columns, 1 column for each asset. The `screen` parameter had caused the pipeline to return a single asset for GOOG, but the calculation was still made for the entire list of assets. This would create a huge bottleneck for more complex factors e.g. calculating growth through linear regression.

In the code cell below, we added `mask=u` that filtered out the assets not listed in through the `StaticAssets` class instantiation:

In [5]:
bogus = BogusFactor(window_length=100, mask=u)
pipe = Pipeline(
    columns={
        'b': bogus
    },
    screen=u
)
mdf = run_pipeline(pipe, start_date, end_date)

(100, 1)


As shown in the output above, there is only a single asset in the calculation.